In [21]:
import cv2
import numpy as np

In [22]:
def load_images(image_path1, image_path2):
    """Load the two images."""
    img1 = cv2.imread(image_path1, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(image_path2, cv2.IMREAD_GRAYSCALE)
    if img1 is None or img2 is None:
        raise ValueError("One of the images didn't load. Check file paths.")
    return img1, img2

In [23]:
def detect_and_match_features(img1, img2):
    """Detect and match features between two images using ORB."""
    # Initiate ORB detector
    orb = cv2.ORB_create()

    # Find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    # Create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Match descriptors
    matches = bf.match(des1, des2)

    # Sort them in the order of their distance
    sorted_matches = sorted(matches, key=lambda x: x.distance)

    # Return the top N matches
    return kp1, kp2, sorted_matches[:30]


In [24]:
def calculate_disparity(kp1, kp2, matches):
    """Calculate average disparity between matched keypoints."""
    disparities = []
    for match in matches:
        # Query index is index in kp1, train index is index in kp2
        x1, x2 = kp1[match.queryIdx].pt[0], kp2[match.trainIdx].pt[0]
        disparities.append(x1 - x2)
    return np.mean(disparities)


In [25]:
def compute_depth(focal_length, baseline, disparity):
    """Compute the depth based on the disparity."""
    if disparity == 0:
        return float('inf')  # Avoid division by zero
    return (focal_length * baseline) / abs(disparity)

In [26]:
# Parameters
focal_length = 1495  # Focal length of the camera in pixels
baseline = 100       # Distance between the two camera centers in mm

# Load images
img1, img2 = load_images('image1.jpg', 'image2.jpg')

# Detect and match features
keypoints1, keypoints2, matches = detect_and_match_features(img1, img2)

# Calculate disparity
average_disparity = calculate_disparity(keypoints1, keypoints2, matches)

# Compute depth
depth = compute_depth(focal_length, baseline, average_disparity)

print("Average Disparity: {:.2f} pixels".format(average_disparity))
print("Estimated Depth: {:.2f} mm".format(depth))

# Visualization of matches
img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imshow("Matches between images", img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

Average Disparity: 678.68 pixels
Estimated Depth: 220.28 mm
